Connect to google drive(for downloading photos)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os

path = "/content/drive/MyDrive/photos"
# Check if file exists
if os.path.isfile(path):
  os.mkdir(path)

Configure ngrok

In [ ]:
import os
!pip install pyngrok
from pyngrok import ngrok

port = 5000

ngrok_token =  os.getenv('NGROK_TOKEN')

#Setting an auth token allows us to open multiple tunnels at the same time
ngrok.set_auth_token(ngrok_token)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

FLASK(download photos or sending echo message)

In [ ]:
import requests
from flask import Flask, request
import threading

app = Flask(__name__)

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = "https" + public_url.split('http')[1]
print(app.config["BASE_URL"])

token = os.getenv('BOT_TOKEN')
bot = Bot(token, app.config["BASE_URL"])
drive_dir = '/content/drive/MyDrive/'

approve_clients = []

db = Database()

def download_photo(bot, message, download_path=None):
    if "photo" in message:
      photo_path, downloaded_photo = bot.get_photo(message["photo"][0]['file_id'])
    elif "document" in message:
      photo_path, downloaded_photo = bot.get_photo(message["document"]["thumb"]['file_id'])
      photo_path = "photos" + photo_path.split("thumbnails")[1]
    else:
      return ""
    if download_path is None:
      download_path = drive_dir + photo_path
    downloaded_photo.save(download_path)
    return download_path

@app.route("/", methods=["GET", "POST"])
def receive_update():
  if request.method == "POST":
      print(request.json)
      message = request.json["message"]
      chat_id = message["chat"]["id"]
      photos = message.get("photo")
      uid = message["from"]["id"]

      # /my_sign + photo - functionality
      if ("caption_entities" in message and message["caption"] == "/my_sign" and message["caption_entities"][0]["type"] == "bot_command" and
          ("photo" in message) or ("document" in message and message["document"]["mime_type"].split("/")[0] == "image") ) or (uid  in approve_clients):

        dpath = download_photo(bot, message, drive_dir + "photos/" + str(uid) + ".jpg")

        # check, if uid exists in db
        # if exists
        if (uid not in approve_clients) and (db.get_sign_by_uid(uid) != []):
          bot.send_message(chat_id, "Do you want to change your signature? (yes/no)")
          approve_clients.append(uid)

        else:
        # check if answer is "no"
          if "text" in message and message["text"] == "no" and uid in approve_clients:
            approve_clients.remove(uid)
            dpath = drive_dir + "photos/" + str(uid) + ".jpg"
            
            # Delete photo after processing
            os.remove(dpath)
            bot.send_message(chat_id, "Ok")
        #if answer is "yes" or it's new user> then
          elif ("text" in message and message["text"] == "yes") or db.get_sign_by_uid(uid)== []:
            if db.get_sign_by_uid(uid)!= [] :
              db.delete_sign_by_uid(uid)
              approve_clients.remove(uid)
              dpath = drive_dir + "photos/" + str(uid) + ".jpg"

            embedding = create_embedding(dpath)

            db.insert_sign(embedding, uid)
            
          # Delete photo after processing
            os.remove(dpath)
            bot.send_message(chat_id, "Success! Now you can verify signature")

      elif ("photo" in message) or ("document" in message and message["document"]["mime_type"].split("/")[0] == "image"):
        if (db.get_sign_by_uid(uid) == []):
            bot.send_message(chat_id, "Oof, can you send photo/scan of your signature with caption - /my_sign ?")
        else:

          dpath = download_photo(bot, message)

          res = predict(dpath)
          print(res)

          #compare embeddings
          embedding = create_embedding(dpath)
          print(embedding, db.get_sign_by_uid(uid))
          print(compare_embeddings(embedding, db.get_sign_by_uid(uid)))
          ## TODO..

          # Delete photo after processing
          os.remove(dpath)

          bot.send_message(chat_id, res)

      # Start command
      elif "entities" in message and message["text"] == "/start" and message["entities"][0]["type"] == "bot_command":
          bot.send_message(chat_id, "Hello, %s! Can you send me scan of your signature?" % (message["from"]["first_name"]))
      # Otherwise
      else:
          bot.send_message(chat_id, "It's not a photo of signature")

  return message

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
